## Attatchment management of e-mails in Gmail

https://developers.google.com/gmail/api/quickstart/python

In [ ]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import base64
import email
from apiclient import errors
import datetime

In [ ]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

In [ ]:
store_dir = "your/path/"
user_id="me"
query = "from:someone@somewhere.org"

In [ ]:
"""Shows basic usage of the Gmail API.
Downloads attachments from specified e-mails.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server()
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('gmail', 'v1', credentials=creds)

try:
    response = service.users().messages().list(userId=user_id,q=query).execute()
    messages = []
    if 'messages' in response:
        messages.extend(response['messages'])

    msg_ids = [messages[x]["id"] for x in range(len(messages))]

    for msg_id in msg_ids:

        message = service.users().messages().get(userId=user_id, id=msg_id, format="full", metadataHeaders=None).execute()
        headers=message["payload"]["headers"]
        subject= [i['value'] for i in headers if i["name"]=="Subject"]
        print(subject)
        snippet = message['snippet']

        for part in message['payload']['parts']:
            if part['filename']:
                if 'data' in part['body']:
                    data=part['body']['data']
                    filename_raw = part['filename']
                else:
                    att_id=part['body']['attachmentId']
                    att=service.users().messages().attachments().get(userId=user_id, messageId=msg_id,id=att_id).execute()
                    data=att['data']
                    filename_raw = part['filename']
                file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))

        # Datei Beschriftung
        s = int(message["internalDate"]) / 1000
        inboxDate = datetime.datetime.fromtimestamp(s).strftime('%Y-%m-%d')
        reciptFrom = query[query.find("@")+1:query.find(".")]
        filename = str(inboxDate + "_" + reciptFrom + "_" + filename_raw)

        # Speicherpfad erstellen
        if not os.path.exists(str(store_dir + "/" +  reciptFrom + "/")):
            os.makedirs(str(store_dir + "/" +  reciptFrom + "/"))

        save_path = str(store_dir + "/" +  reciptFrom + "/")
        completeName = os.path.join(save_path , filename)

        print(completeName)
        with open(completeName, 'wb') as f:
                    f.write(file_data)

except errors.HttpError:
    print('An error occurred: {}').format(errors)
    next

if not msg_ids:
    print('No E-Mails found.')